# Optimized Polymer Property Prediction: SMILES->RDKIT->LGBM->FTW 🧪⚡🚀

**Optimization Highlights:**
- Enhanced feature engineering with Morgan fingerprints
- Hyperparameter optimization using Optuna
- Advanced feature selection techniques
- Improved data preprocessing pipeline
- Target-specific model configurations

In [1]:
# Install RDKit for offline use
!pip install /kaggle/input/rdkit-install-whl/rdkit_wheel/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install optuna

Processing /kaggle/input/rdkit-install-whl/rdkit_wheel/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_regression
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import optuna
import warnings
import joblib
from tqdm import tqdm

warnings.filterwarnings('ignore')

# Enhanced Feature Engineering
- Added Morgan fingerprints for structural information
- Custom feature selection pipeline
- Advanced data preprocessing

In [3]:
def get_molecular_descriptors(max_autocorr=30):
    """Get optimized molecular descriptors with filtering"""
    descriptor_list = []
    test_mol = Chem.MolFromSmiles('CCO')
    
    # Core descriptors that are most informative
    core_descriptors = [
        'MolWt', 'MolLogP', 'NumRotatableBonds', 'NumAromaticRings', 
        'BertzCT', 'TPSA', 'qed', 'FractionCSP3', 'HeavyAtomCount',
        'NHOHCount', 'NOCount', 'NumAliphaticRings', 'NumHeteroatoms',
        'NumSaturatedRings', 'RingCount', 'HallKierAlpha'
    ]
    
    # Add core descriptors
    for name in core_descriptors:
        try:
            func = getattr(Descriptors, name)
            descriptor_list.append((name, func))
        except:
            pass
    
    # Add limited AUTOCORR2D descriptors
    autocorr_descriptors = []
    for i in range(1, max_autocorr+1):
        name = f'AUTOCORR2D_{i}'
        try:
            func = getattr(Descriptors, name)
            autocorr_descriptors.append((name, func))
        except:
            pass
    
    descriptor_list += autocorr_descriptors
    print(f"✅ Selected {len(descriptor_list)} descriptors")
    return descriptor_list

In [4]:
def smiles_to_features(smiles_list, molecular_descriptors, morgan_bits=512, radius=2):
    """Enhanced feature extraction with Morgan fingerprints"""
    features = []
    total = len(smiles_list)
    
    print(f"Processing {total} SMILES with Morgan fingerprints...", end="", flush=True)
    
    for i, smiles in enumerate(tqdm(smiles_list, total=total)):
        mol_features = []
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                # Fill with NaN for descriptors and zeros for fingerprints
                mol_features = [np.nan] * len(molecular_descriptors) + [0] * morgan_bits
            else:
                # Calculate descriptors
                for name, func in molecular_descriptors:
                    try:
                        value = func(mol)
                        if np.isinf(value) or abs(value) > 1e10:
                            value = np.nan
                        mol_features.append(value)
                    except:
                        mol_features.append(np.nan)
                
                # Calculate Morgan fingerprints
                try:
                    fp = AllChem.GetMorganFingerprintAsBitVect(
                        mol, radius=radius, nBits=morgan_bits, useChirality=True
                    )
                    fp_list = list(fp.ToBitString())
                    mol_features.extend([int(bit) for bit in fp_list])
                except:
                    mol_features.extend([0] * morgan_bits)
        except:
            mol_features = [np.nan] * len(molecular_descriptors) + [0] * morgan_bits
        
        features.append(mol_features)
    
    features = np.array(features, dtype=float)
    print(" ✅", flush=True)
    return features

In [5]:
def feature_selection_pipeline(X, y, feature_names, k=200):
    """Advanced feature selection pipeline"""
    print("🔍 Running feature selection pipeline...")
    
    # Step 1: Remove low variance features
    selector_vt = VarianceThreshold(threshold=0.01)
    X_vt = selector_vt.fit_transform(X)
    selected_mask_vt = selector_vt.get_support()
    print(f"  VarianceThreshold: {sum(selected_mask_vt)} features retained")
    
    # Step 2: Mutual information selection
    selector_mi = SelectKBest(score_func=mutual_info_regression, k=min(k, X_vt.shape[1]))
    X_mi = selector_mi.fit_transform(X_vt, y)
    selected_mask_mi = selector_mi.get_support()
    
    # Combine selection masks
    final_mask = selected_mask_vt.copy()
    final_mask[final_mask] = selected_mask_mi
    
    print(f"  MutualInfo: {sum(final_mask)} features selected")
    
    # Get selected feature names
    selected_features = [feature_names[i] for i in range(len(feature_names)) if final_mask[i]]
    
    return X[:, final_mask], selected_features

In [6]:
def clean_features(X, feature_names=None):
    """Enhanced feature cleaning with robust scaling"""
    # Create a copy to avoid modifying original
    X_clean = X.copy()
    
    # Replace inf with NaN
    X_clean[np.isinf(X_clean)] = np.nan
    
    # Impute missing values with median
    for i in range(X_clean.shape[1]):
        col = X_clean[:, i]
        if np.isnan(col).any():
            median = np.nanmedian(col)
            X_clean[np.isnan(col), i] = median if not np.isnan(median) else 0
    
    # Apply robust scaling
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X_clean)
    
    return X_scaled, scaler

# Hyperparameter Optimization with Optuna

In [7]:
def objective(trial, X, y):
    """Optuna objective function for hyperparameter tuning"""
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 32, 512),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'verbose': -1,
        'random_state': 42
    }
    
    # 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []
    
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
        
        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=2000,
            callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
        )
        
        val_pred = model.predict(X_val)
        cv_scores.append(mean_absolute_error(y_val, val_pred))
    
    return np.mean(cv_scores)

# Enhanced Model Training
- Target-specific hyperparameters
- Advanced feature selection
- Optuna for hyperparameter tuning

In [8]:
def train_target_property(X_target, y_target, target_name):
    """Enhanced training with hyperparameter optimization"""
    print(f"📊 Training {target_name} on {len(y_target)} samples")
    print(f"📈 Target range: {y_target.min():.4f} to {y_target.max():.4f}")
    
    # Target-specific hyperparameter tuning
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X_target, y_target), n_trials=30)
    
    best_params = study.best_params
    print(f"🎯 Best params for {target_name}: {best_params}")
    
    # Add fixed parameters
    best_params.update({
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'verbose': -1,
        'random_state': 42
    })
    
    # 7-fold cross-validation
    cv_scores = []
    models = []
    all_val_true = []
    all_val_pred = []
    
    kf = KFold(n_splits=7, shuffle=True, random_state=42)
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_target)):
        X_train, X_val = X_target[train_idx], X_target[val_idx]
        y_train, y_val = y_target[train_idx], y_target[val_idx]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
        
        model = lgb.train(
            best_params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=5000,
            callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
        )
        
        val_pred = model.predict(X_val)
        cv_score = mean_absolute_error(y_val, val_pred)
        cv_scores.append(cv_score)
        models.append(model)
        
        all_val_true.extend(y_val)
        all_val_pred.extend(val_pred)
        
        print(f"----Fold {fold+1} MAE = {cv_score:.4f}", flush=True)
    
    cv_mean = np.mean(cv_scores)
    print(f"===CV MAE: {cv_mean:.4f} ± {np.std(cv_scores):.3f}===")
    return models, cv_mean, all_val_true, all_val_pred

# Data Loading & Preparation

In [9]:
# Load data
train_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')

# Define targets
targets = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
print("Target counts:")
print(train_df[targets].count())

# Get molecular descriptors
molecular_descriptors = get_molecular_descriptors(max_autocorr=30)
descriptor_names = [name for name, _ in molecular_descriptors]

# Generate feature names (descriptors + morgan fingerprints)
morgan_bits = 512
feature_names = descriptor_names + [f"Morgan_{i}" for i in range(morgan_bits)]

# Extract features for training data
print("\nGenerating features for training data...")
X_raw = smiles_to_features(
    train_df['SMILES'].values, 
    molecular_descriptors, 
    morgan_bits=morgan_bits
)

# Clean and scale features
X, scaler_all = clean_features(X_raw)
print(f"Total features: {X.shape[1]}")

Target counts:
Tg          511
FFV        7030
Tc          737
Density     613
Rg          614
dtype: int64
✅ Selected 46 descriptors

Generating features for training data...
Processing 7973 SMILES with Morgan fingerprints...

100%|██████████| 7973/7973 [00:47<00:00, 166.58it/s]


 ✅
Total features: 558


# Model Training with Feature Selection

In [10]:
# Store trained models
trained_models = {}
cv_scores = []
all_cv_predictions = {}
all_cv_true = {}

# Train each target with feature selection
for target in targets:
    print(f"\n{'='*50}")
    print(f"Training {target}...")
    
    # Prepare training data
    mask = train_df[target].notna()
    X_target = X[mask]
    y_target = train_df[target].values[mask]
    
    # Apply feature selection for this target
    X_selected, selected_features = feature_selection_pipeline(
        X_target, y_target, feature_names, k=200
    )
    print(f"Selected {len(selected_features)} features for {target}")
    
    # Train models
    models, cv_score, val_true, val_pred = train_target_property(
        X_selected, y_target, target
    )
    
    # Store results
    trained_models[target] = {
        'models': models,
        'selected_features': selected_features,
        'feature_indices': [feature_names.index(f) for f in selected_features]
    }
    cv_scores.append(cv_score)
    all_cv_true[target] = val_true
    all_cv_predictions[target] = val_pred
    print(f"{'='*50}\n")


Training Tg...
🔍 Running feature selection pipeline...
  VarianceThreshold: 502 features retained


[I 2025-08-07 05:02:39,960] A new study created in memory with name: no-name-843aa851-98f3-492d-98f2-f01549a34792


  MutualInfo: 200 features selected
Selected 200 features for Tg
📊 Training Tg on 511 samples
📈 Target range: -148.0297 to 472.2500
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[750]	valid_0's l1: 51.0455
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1308]	valid_0's l1: 59.4949
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1309]	valid_0's l1: 54.3807
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[612]	valid_0's l1: 48.1239
Training until validation scores don't improve for 100 rounds


[I 2025-08-07 05:02:42,367] Trial 0 finished with value: 51.63361117531756 and parameters: {'num_leaves': 263, 'learning_rate': 0.010913738519659577, 'feature_fraction': 0.9664211466939991, 'bagging_fraction': 0.7202131592350449, 'bagging_freq': 6, 'lambda_l1': 3.1755412822837012, 'lambda_l2': 3.936403787827294, 'min_data_in_leaf': 53, 'max_depth': 6, 'min_child_samples': 25}. Best is trial 0 with value: 51.63361117531756.


Early stopping, best iteration is:
[731]	valid_0's l1: 45.1231
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[580]	valid_0's l1: 55.0346
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1960]	valid_0's l1: 57.5022
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1170]	valid_0's l1: 51.7733
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[677]	valid_0's l1: 46.5933
Training until validation scores don't improve for 100 rounds


...

Early stopping, best iteration is:
[153]	valid_0's l1: 1.59364
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[219]	valid_0's l1: 1.77155
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[288]	valid_0's l1: 1.70399
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[549]	valid_0's l1: 1.91629
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[437]	valid_0's l1: 1.5092
Training until validation scores don't improve for 100 rounds


[I 2025-08-07 05:13:55,912] Trial 28 finished with value: 1.7085051948599468 and parameters: {'num_leaves': 36, 'learning_rate': 0.02273920173078366, 'feature_fraction': 0.8707217425429269, 'bagging_fraction': 0.6413137640398514, 'bagging_freq': 3, 'lambda_l1': 1.7117253521227471, 'lambda_l2': 6.882565891446131, 'min_data_in_leaf': 22, 'max_depth': 12, 'min_child_samples': 15}. Best is trial 23 with value: 1.622520671984605.


Early stopping, best iteration is:
[549]	valid_0's l1: 1.64149
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[468]	valid_0's l1: 1.91781
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[468]	valid_0's l1: 1.95518
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[822]	valid_0's l1: 2.09985
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[486]	valid_0's l1: 1.69717
Training until validation scores don't improve for 100 rounds


[I 2025-08-07 05:13:57,758] Trial 29 finished with value: 1.8982341665115285 and parameters: {'num_leaves': 310, 'learning_rate': 0.012505341576403744, 'feature_fraction': 0.9259019623067175, 'bagging_fraction': 0.5468300502770282, 'bagging_freq': 6, 'lambda_l1': 9.415586973663466, 'lambda_l2': 5.693119651742278, 'min_data_in_leaf': 45, 'max_depth': 12, 'min_child_samples': 78}. Best is trial 23 with value: 1.622520671984605.


Early stopping, best iteration is:
[853]	valid_0's l1: 1.82116
🎯 Best params for Rg: {'num_leaves': 72, 'learning_rate': 0.02667079291204184, 'feature_fraction': 0.724786880184618, 'bagging_fraction': 0.563451498668369, 'bagging_freq': 4, 'lambda_l1': 1.0095685091767632, 'lambda_l2': 8.528542270030172, 'min_data_in_leaf': 5, 'max_depth': 14, 'min_child_samples': 16}
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[149]	valid_0's l1: 1.76594
----Fold 1 MAE = 1.7659
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[163]	valid_0's l1: 1.72821
----Fold 2 MAE = 1.7282
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[176]	valid_0's l1: 1.57839
----Fold 3 MAE = 1.5784
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[224]	valid_0's l1: 1.78638
----Fold 4 MAE = 1.7864
Training until validation scores do

# Performance Evaluation

In [11]:
# Import competition metric
import open_polymer_2025_metric as metric

# Calculate competition scores
competition_scores = []
for target in targets:
    comp_score = metric.scaling_error(
        np.array(all_cv_true[target]), 
        np.array(all_cv_predictions[target]), 
        target
    )
    competition_scores.append(comp_score)

# Calculate overall competition score
cv_true_df = pd.DataFrame()
cv_pred_df = pd.DataFrame()

for target in targets:
    # Pad arrays to same length
    max_len = max(len(all_cv_true[t]) for t in targets)
    true_padded = list(all_cv_true[target]) + [metric.NULL_FOR_SUBMISSION] * (max_len - len(all_cv_true[target]))
    pred_padded = list(all_cv_predictions[target]) + [metric.NULL_FOR_SUBMISSION] * (max_len - len(all_cv_predictions[target]))
    
    cv_true_df[target] = true_padded
    cv_pred_df[target] = pred_padded

# Add dummy id column
cv_true_df['id'] = range(len(cv_true_df))
cv_pred_df['id'] = range(len(cv_pred_df))

# Calculate overall score
estimated_lb_score = metric.score(cv_true_df, cv_pred_df, 'id')

print("=" * 50)
print(f"Trained models for {len(targets)} targets")
print(f"Average CV MAE: {np.mean(cv_scores):.4f}")
print(f"Individual competition scores: {[f'{s:.4f}' for s in competition_scores]}")
print(f"🎯 ESTIMATED LB SCORE: {estimated_lb_score:.4f}")
print("=" * 50)

Trained models for 5 targets
Average CV MAE: 10.3263
Individual competition scores: ['0.0805', '0.0139', '0.0590', '0.0319', '0.0648']
🎯 ESTIMATED LB SCORE: 0.0564


# Prediction & Submission

In [12]:
def predict_target_property(test_df, target_name, model_info, molecular_descriptors, morgan_bits=512):
    """Enhanced prediction function"""
    print(f"PREDICTING: {target_name}")
    
    # Generate raw features
    X_test_raw = smiles_to_features(
        test_df['SMILES'].values, 
        molecular_descriptors, 
        morgan_bits=morgan_bits
    )
    
    # Clean features
    X_test, _ = clean_features(X_test_raw)
    
    # Select the same features used in training
    feature_indices = model_info['feature_indices']
    X_test_selected = X_test[:, feature_indices]
    
    # Average predictions from all models
    predictions = np.zeros(len(test_df))
    for model in model_info['models']:
        pred = model.predict(X_test_selected)
        predictions += pred
    
    predictions /= len(model_info['models'])
    print(f"📊 Predictions range: {predictions.min():.4f} to {predictions.max():.4f}")
    
    return predictions

In [13]:
# Generate predictions for all targets
all_predictions = {}
for target in targets:
    predictions = predict_target_property(
        test_df, target, trained_models[target], molecular_descriptors
    )
    all_predictions[target] = predictions

# Create submission
submission = pd.DataFrame({'id': test_df['id']})
for target in targets:
    submission[target] = all_predictions[target]

submission.to_csv('submission.csv', index=False)

print("\n✅ Submission saved:")
print(submission.head().to_string(index=False, float_format='%.4f'))

PREDICTING: Tg
Processing 3 SMILES with Morgan fingerprints...

100%|██████████| 3/3 [00:00<00:00, 131.92it/s]

 ✅
📊 Predictions range: 30.9414 to 159.1035
PREDICTING: FFV
Processing 3 SMILES with Morgan fingerprints...


100%|██████████| 3/3 [00:00<00:00, 114.76it/s]

 ✅
📊 Predictions range: 0.3456 to 0.3772
PREDICTING: Tc
Processing 3 SMILES with Morgan fingerprints...

...

 ✅


📊 Predictions range: 0.9576 to 1.3142
PREDICTING: Rg
Processing 3 SMILES with Morgan fingerprints...

100%|██████████| 3/3 [00:00<00:00, 112.50it/s]

 ✅


📊 Predictions range: 16.3448 to 18.4779

✅ Submission saved:
        id       Tg    FFV     Tc  Density      Rg
1109053969  30.9414 0.3456 0.1534   1.3142 18.4779
1422188626 159.1035 0.3772 0.2709   0.9576 16.3448
2032016830  71.2914 0.3601 0.2872   1.0553 18.3186
